# Download data

In [2]:
!gdown http://headkraken.gg/soft/Dan/LCB_data_snd_devided.zip
!unzip -qq LCB_data_snd_devided.zip

Downloading...
From: http://headkraken.gg/soft/Dan/LCB_data_snd_devided.zip
To: /content/LCB_data_snd_devided.zip
100% 8.15G/8.15G [07:57<00:00, 17.0MB/s]


In [3]:
!gdown http://headkraken.gg/soft/LCB_demos.zip
!unzip -qq LCB_demos.zip

Downloading...
From: http://headkraken.gg/soft/LCB_demos.zip
To: /content/LCB_demos.zip
100% 324M/324M [00:35<00:00, 9.16MB/s]


In [4]:
!gdown --id 13s0uq6eWU5bx3Ctik5Wy4baN-wBctoOu
!unzip -q LCB_added_rep.zip

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=13s0uq6eWU5bx3Ctik5Wy4baN-wBctoOu
To: /content/LCB_added_rep.zip
100% 294k/294k [00:00<00:00, 77.6MB/s]


In [5]:
!gdown --id 1BdXSmAlogOrJyyoQ9SQ1HzOZpTvCBrw-

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1BdXSmAlogOrJyyoQ9SQ1HzOZpTvCBrw-
To: /content/demo_analysis_new.py
100% 8.29k/8.29k [00:00<00:00, 12.6MB/s]


In [6]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Prepare data

In [7]:
import warnings
warnings.filterwarnings("ignore")

from data_prep import *

In [8]:
file_path='./LCB_added'
path_to_audio='./LCB_data_snd_devided'
path_to_splitted_audio='./LCB_audio'

In [9]:
train_dataloader,val_dataloader = get_dataloader(file_path=file_path,
                                                 path_to_audio=path_to_audio,
                                                 path_to_splitted_audio=path_to_splitted_audio,
                                                 test_size=0.2,
                                                 use_game_context=True,
                                                 batch_size=32)

Start splitting audio to  ./LCB_audio
Finish splitting

Emotion statistic:
1 - 4686     интерес
2 - 473     радость
3 - 170     удивление
4 - 173     горе
5 - 632     гнев
6 - 6     отвращение
7 - 158     презрение
8 - 265     страх
9 - 1     стыд
10 - 67     вина

Prepare train and val lists
It took:  5.29  minutes
train size:  5304
val size:  1326

Prepate train dataset
Prepate val dataset


In [21]:
next(iter(train_dataloader))[0].shape

torch.Size([32, 1, 66150])

In [11]:
next(iter(train_dataloader))[1].shape

torch.Size([32, 12])

In [12]:
next(iter(train_dataloader))[2].shape

torch.Size([32, 10])

In [17]:
#for ML methods
x_train,ctx_train,y_train,x_test,ctx_test,y_test = get_train_test(file_path=file_path,
                                                 path_to_audio=path_to_audio,
                                                 path_to_splitted_audio=path_to_splitted_audio,
                                                 test_size=0.2,
                                                 use_game_context=True)

Start splitting audio to  ./LCB_audio
Finish splitting

Emotion statistic:
1 - 4686     интерес
2 - 473     радость
3 - 170     удивление
4 - 173     горе
5 - 632     гнев
6 - 6     отвращение
7 - 158     презрение
8 - 265     страх
9 - 1     стыд
10 - 67     вина

Prepare train and val lists
It took:  5.37  minutes
train size:  5304
val size:  1326
Prepate train dataset
Prepate test dataset


In [19]:
x_train.shape,ctx_train.shape,y_train.shape

((5304, 66150), (5304, 12), (5304, 10))